In [204]:
import numpy as np
import csv
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier


In this script, we have three ML model:
SVC: C-Support Vector Classification.
SGDClassifier: Linear classifiers (SVM, logistic regression, a.o.) with SGD training.
This estimator implements regularized linear models with stochastic gradient descent (SGD) learning: the gradient of the loss is estimated each sample at a time and the model is updated along the way with a decreasing strength schedule (aka learning rate). SGD allows minibatch (online/out-of-core) learning, see the partial_fit method. For best results using the default learning rate schedule, the data should have zero mean and unit variance.

Random Forest:
RF gives the best result:  f1: 0.97345133  recall: 0.84210526


In [2]:
class myQuote:
    def __init__(self, text):
        #read in CSV, process
        self.quoteText = text
        self.quoteID = hash(self.quoteText)
        self.link = []
        self.name = []
        self.description = []
        self.scores = []
        self.cos = []
    
    def __hash__(self):
        return self.quoteID
    
    def __str__(self):
        return "Object text: " + self.quoteText + '\n' +\
        "Links: " + str(self.link) + '\n' +\
        "Names: " + str(self.name) + '\n' +\
        "Description " + str(self.description) + '\n' +\
        "Scores: " + str(self.scores)
        

In [3]:
infile = open('searchResults','rb')
results = pickle.load(infile, encoding='bytes')

In [4]:
#get the first 200 labels: those are the one in the mannual annotate sample
count = 0
proto_matrix = []
for item in results:
    #results[item].scores[0]
    row = results[item].scores
    row = np.append(row, results[item].cos)
    proto_matrix.append(row)
    count += 1
    if count > 199:
        break

matrix = np.matrix(proto_matrix)

In [ ]:
#read labels from ma

In [5]:
labels = pd.read_csv('MLData_recode_mannual.csv')
label = labels['label'].values
label = label.astype(np.float)
print(len(label))
print(type(label))

200
<class 'numpy.ndarray'>


In [401]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_matrix = scaler.fit_transform(matrix)


In [402]:
#split train test
X = scaled_matrix
y = label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [477]:
#####grid search (the parameters predict everything to one class, we should use a separated 
#sample for tuning parameters, but not enough cases so far)
svc = svm.SVC()
parameters = [{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': [0.01, 0.001, 0.0001],
                     'C': [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.0,3.0], 'class_weight':['balanced']}]
                   
grid_search_item = GridSearchCV(estimator = svc,
                          param_grid = parameters,
                           scoring = 'accuracy',
                           n_jobs = -1)
grid_search = grid_search_item.fit(X_train, y_train)

print(grid_search.best_score_)####not sure how come this is so good,but the parameters don't work good in the model
print(grid_search.best_params_)

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
0.888060 (0.016236) with: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 0.01, 'kernel': 'sigmoid'}

0.917910447761
{'C': 1.7, 'class_weight': 'balanced', 'gamma': 0.01, 'kernel': 'sigmoid'}
0.910448 (0.032753) with: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 0.01, 'kernel': 'linear'}
0.835821 (0.007038) with: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 0.01, 'kernel': 'poly'}
0.902985 (0.051999) with: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 0.01, 'kernel': 'rbf'}
0.888060 (0.016236) with: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 0.01, 'kernel': 'sigmoid'}
0.910448 (0.032753) with: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 0.001, 'kernel': 'linear'}
0.388060 (0.316693) with: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 0.001, 'kernel': 'poly'}
0.388060 (0.316693) with: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 0.001, 'kernel': 'rbf'}
0.388060 (0.316693) with: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 0.001, 'kernel': 'sigmoid'}
0.910448 (0.032753) with: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 0.0001, 'kernel': 'linear'}
0.388060 (0.

In [520]:
svc = svm.SVC(gamma=0.01, class_weight='balanced', C = 0.5, kernel = 'linear')
svc.fit(X_train, y_train) 

SVC(C=0.5, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [521]:
y = svc.predict(X_test)
y

array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [522]:
y_test

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
        0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.])

In [523]:
cm = confusion_matrix(y_test, y)
print(cm)
print(f1_score(y_test, y, average= None) )

[[54  2]
 [ 3  7]]
[ 0.95575221  0.73684211]


####SGDClassifier 

In [407]:
#####grid search (the parameters predict everything to one class, we should use a separated 
#sample for tuning parameters, but not enough cases so far)
svc = SGDClassifier(max_iter= 1000, alpha=0.05)
parameters = [{'alpha': [0.01, 0.05, 0.001, 0.005], 'class_weight':[{1:5, 1:10, 1:15}]}]
                   
grid_search_item = GridSearchCV(svc,
                                cv=5,
                          param_grid = parameters,
                           scoring = 'accuracy',
                           n_jobs = -1)
grid_search = grid_search_item.fit(X_train, y_train)

print(grid_search.best_score_)####not sure how come this is so good,but the parameters don't work good in the model
print(grid_search.best_params_)

0.880597014925
{'alpha': 0.001, 'class_weight': {1: 15}}


In [408]:
clf = SGDClassifier(max_iter=1000, alpha=0.001, loss="hinge")
clf.fit(X_train, y_train) 

SGDClassifier(alpha=0.001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=1000, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [409]:
y = clf.predict(X_test)
y

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [410]:
y_test

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
        0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.])

In [411]:
print(f1_score(y_test, y, average= None))
cm = confusion_matrix(y_test, y)
print(cm)

[ 0.95726496  0.66666667]
[[56  0]
 [ 5  5]]


In [ ]:
########recode data

In [122]:
file  = pd.read_csv('MLData.csv')
file['label'][1:5]

1       quote
2       quote
3       quote
4    NotQuote
Name: label, dtype: object

In [125]:
def recode(series):
    if 'quote' in series:
        return 1
    else:
        return 0
    
file['label'] = file['label'].apply(recode)

In [127]:
print(file['label'][1:5])
file.to_csv('MLData_recode.csv')

1    1
2    1
3    1
4    0
Name: label, dtype: int64


###random forest

In [412]:
rf = RandomForestClassifier()

In [536]:
#####grid search (the parameters predict everything to one class, we should use a separated 
#sample for tuning parameters, but not enough cases so far)

parameters = [{'max_features':['auto','sqrt','log2'], 'class_weight':['balanced'], 
               'max_leaf_nodes':[10,50,100], 'max_depth':[2,5,10,20], 'n_estimators' : [50,100,200,300,400]}]
                   
grid_search_item = GridSearchCV(rf,
                          param_grid = parameters,
                           scoring = 'accuracy',
                           n_jobs = -1)
grid_search = grid_search_item.fit(X_train, y_train)

print(grid_search.best_score_)####not sure how come this is so good,but the parameters don't work good in the model
print(grid_search.best_params_)

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.970149253731
{'class_weight': 'balanced', 'max_depth': 2, 'max_features': 'auto', 'max_leaf_nodes': 10, 'n_estimators': 100}
0.955224 (0.019032) with: {'class_weight': 'balanced', 'max_depth': 2, 'max_features': 'auto', 'max_leaf_nodes': 10, 'n_estimators': 50}
0.970149 (0.010918) with: {'class_weight': 'balanced', 'max_depth': 2, 'max_features': 'auto', 'max_leaf_nodes': 10, 'n_estimators': 100}
0.955224 (0.032259) with: {'class_weight': 'balanced', 'max_depth': 2, 'max_features': 'auto', 'max_leaf_nodes': 10, 'n_estimators': 200}
0.947761 (0.028343) with: {'class_weight': 'balanced', 'max_depth': 2, 'max_features': 'auto', 'max_leaf_nodes': 10, 'n_estimators': 300}
0.955224 (0.032259) with: {'class_weight': 'balanced', 'max_depth': 2, 'max_features': 'auto', 'max_leaf_nodes': 10, 'n_estimators': 400}
0.955224 (0.019032) with: {'class_weight': 'balanced', 'max_depth': 2, 'max_features': 'auto', 'max_leaf_nodes': 50, 'n_estimators': 50}
0.970149 (0.010918) with: {'class_weight': 'bal

In [531]:
rf = RandomForestClassifier(n_estimators=100, max_depth=2, class_weight = 'balanced',
                             random_state=0, max_leaf_nodes = 10, max_features = 'auto')

In [532]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=2, max_features='auto',
            max_leaf_nodes=10, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [533]:
print(rf.feature_importances_)

[ 0.05360163  0.15989177  0.11956025  0.0061933   0.06120305  0.02051895
  0.03359439  0.01085205  0.02192883  0.03351403  0.47914175]


In [534]:
y = rf.predict(X_test)
y

array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [438]:
y_test

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
        0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.])

In [535]:
print(f1_score(y_test, y, average= None))
cm = confusion_matrix(y_test, y)
print(cm)

[ 0.97345133  0.84210526]
[[55  1]
 [ 2  8]]
